In [1]:
import requests
import pandas as pd
import numpy  as np
import re

from bs4 import BeautifulSoup

 <hr style="height:2px;border-width:0;color:gray;background-color:gray"> 

 <p style = "text-align:center;color:gray;font-size:30px">Extraindo a lista de links</p>

 <hr style="height:2px;border-width:0;color:gray;background-color:gray"> 

In [2]:
links = []
for i in range(1,30):
    url = "https://psprices.com/region-br/games/?sort=subscribers&platform=PS4&page="+str(i)
    print(url)
    print("-------------------------------------------------------------------------------------------")
    # -------------------------------------------------------------------------------------------
    # Criando um objeto com o link de interesse 
    url_obj = requests.get(url)
    # Extraindo o código fonte do link 
    data = url_obj.text
    # lxml - processar XML eHTML com Python 
    soup = BeautifulSoup(data, "lxml")
    # Extraindo todas as tags com <a> 
    tags = soup.find_all("a")
    # Criando uma lista vazia 
    
    # -------------------------------------------------------------------------------------------
    # Extraindo URLs das tags com href em <a> 
    for tag in tags:
        
        # Apendando o prefixo no link
        aux = "https://psprices.com"+str(tag.get("href"))
        # Cria um vetor com os links 
        links.append(aux)

https://psprices.com/region-br/games/?sort=subscribers&platform=PS4&page=1
-------------------------------------------------------------------------------------------
https://psprices.com/region-br/games/?sort=subscribers&platform=PS4&page=2
-------------------------------------------------------------------------------------------
https://psprices.com/region-br/games/?sort=subscribers&platform=PS4&page=3
-------------------------------------------------------------------------------------------
https://psprices.com/region-br/games/?sort=subscribers&platform=PS4&page=4
-------------------------------------------------------------------------------------------
https://psprices.com/region-br/games/?sort=subscribers&platform=PS4&page=5
-------------------------------------------------------------------------------------------
https://psprices.com/region-br/games/?sort=subscribers&platform=PS4&page=6
------------------------------------------------------------------------------------------

In [3]:
# Selecionando só os links com o mesmo começo
links2 = list(filter(lambda x: x.lower().startswith("https://psprices.com/region-br/game/"), links))
links_aux = pd.DataFrame(links2)
# Retirando links invalidos
links_fim  = pd.DataFrame(links_aux[~(links_aux == "https://psprices.comNone")])
links_fim2 = pd.DataFrame(links_fim[~(links_fim == "https://psprices.com/region-br/game/1863959/ps-plus-12-month-membership")])   
links_fim2 = pd.DataFrame(links_fim2[~(links_fim2 == "https://psprices.com/region-br/game/1863963/ps-plus-3-month-membership")])   
links_fim2 = pd.DataFrame(links_fim2[~(links_fim2 == "https://psprices.com/region-br/game/1863960/ps-plus-1-month-membership")])   
links_fim2 = pd.DataFrame(links_fim2[~(links_fim2 == "https://psprices.com/region-br/game/1852967/mass-effect-andromeda-edicao-de-recruta-deluxe")])   


links_fim2 = links_fim2.dropna()
# Primeira coluna todas as linhas
links_fim2.iloc[:,0]

0       https://psprices.com/region-br/game/3200971/ma...
1       https://psprices.com/region-br/game/2439074/se...
2       https://psprices.com/region-br/game/2355781/re...
3       https://psprices.com/region-br/game/1374501/he...
4       https://psprices.com/region-br/game/1839351/ho...
                              ...                        
1039    https://psprices.com/region-br/game/103092/swo...
1040    https://psprices.com/region-br/game/31133/flow...
1041    https://psprices.com/region-br/game/2416495/st...
1042    https://psprices.com/region-br/game/997773/aca...
1043     https://psprices.com/region-br/game/1236724/kona
Name: 0, Length: 1043, dtype: object

 <hr style="height:2px;border-width:0;color:gray;background-color:gray"> 

 <p style = "text-align:center;color:gray;font-size:30px">Extraindo as informações dos games</p>

 <hr style="height:2px;border-width:0;color:gray;background-color:gray"> 

In [4]:
# Função para limpeza das variáveis
def tratvarsite(var_site):  
# --------------------------------------------------    
    # removendo os characteres 
    var_site = re.sub("[^0-9]", "", var_site)
# --------------------------------------------------
    if len(var_site) < 5: 
        # concatenando o 0
        var_site = "0"+var_site
        # Atribuindo .
        var_site2 = var_site[:3]+"."+var_site[3:]
        return var_site2
# --------------------------------------------------
    elif len(var_site) > 2: 
        var_site2 = var_site[:3]+"."+var_site[3] 
        return var_site2
# --------------------------------------------------
    else:
        var_site2 = var_site 
        return var_site2


In [5]:
# Criando um dataframe vazio
df = pd.DataFrame(columns = ["nome", "preco_orig", "preco_atual","nota","psn","menor_preco","data","dlc","desc"])
n = 0

# Loop para extrair as informações do site https://psprices.com/
for links_ in links_fim2.iloc[:,0]:
    # URL de interesse -----------------------
    url    = str(links_)
    r      = requests.get(url)
    pagina = BeautifulSoup(r.text,"lxml")
    # Remover tags html ---------------
    cleanr = re.compile("<.*?>")

    # Extração das informações -----------------------------------------------------------
    nome  = str(pagina.find_all(class_ = "content__game__title"))
    dlc   = str(pagina.find_all(class_ = "badge badge-secondary"))
    dlc2  = re.sub("[^0-9]", "", dlc)
    desc  = re.sub(cleanr,"", str(pagina.find_all(class_ = "content__game__long_desc")))
    # find_all utilizado para encontrar a tag
    # Transformando em character e removendo os characteres especiais .
    preco_orig  = re.sub(cleanr,"", str(pagina.find_all(class_ = "old_price")))
    # rsplit(separator, maxsplit)
    preco_orig  = preco_orig.rsplit(", ", 100)[0]
    preco_atual = re.sub(cleanr,"", str(pagina.find_all(class_ = "current"))) 
    score       = re.sub(cleanr,"", str(pagina.find_all(class_ = "scores__opencritic")))
    preco_menor = str(pagina.find_all(class_ = "text-secondary"))

    # -------------------------------------------------------------------------------------
    preco_menor2 = preco_menor.rsplit("— Release date:", 1)[0]
    preco_PSPlus = preco_menor2.rsplit("PS+:", 1)[1]
    preco_menor3 = preco_menor2.rsplit("PS+:", 1)[0]
    data_lanc    = preco_menor.rsplit("Release date:", 1)[1]

    # -------------------------------------------------------------------------------------
    # Pegando informação entre dois chatacter Data
    start      = data_lanc.find("\n              <strong>") + len("\n              <strong>")
    end        = data_lanc.find("</strong>\n</div>]")
    data_lanc2 = data_lanc[start:end]
    # Pegando informação entre dois chatacter Nome
    start1      = nome.find("\n<h1>") + len("\n<h1>")
    end1        = nome.find("</h1>")
    nome2 = nome[start1:end1]

    # -------------------------------------------------------------------------------------
    # Chamando as funções
    preco_orig2   = tratvarsite(preco_orig)
    preco_atual2  = tratvarsite(preco_atual)

    preco_PSPlus2 = tratvarsite(preco_PSPlus)
    preco_menor32 = tratvarsite(preco_menor3)

    # -------------------------------------------------------------------------------------
    # Guardando as informações em um dataframe
    df.loc[n] = [nome2, preco_orig2, preco_atual2, score, preco_PSPlus2, preco_menor32, data_lanc2, dlc2, desc]
    print(n, end = ",")
    print("Nome do game:     ", nome2)
    n = n + 1

0,Nome do game:      Marvel's Spider-Man: Game Of The Year Edition
1,Nome do game:      Sekiro: Shadows Die Twice
2,Nome do game:      Red Dead Redemption 2
3,Nome do game:      Hellblade: Senua's Sacrifice
4,Nome do game:      Horizon Zero Dawn: Complete Edition
5,Nome do game:      Final Fantasy VII Remake
6,Nome do game:      Star Wars Jedi: Fallen Order
7,Nome do game:      Crash Bandicoot N. Sane Trilogy
8,Nome do game:      God Of War
9,Nome do game:      The Last Guardian
10,Nome do game:      Uncharted: The Lost Legacy
11,Nome do game:      Shadow Of The Tomb Raider Definitive Edition
12,Nome do game:      v class="content__game__title d-block my-0">
<h1 class="mb-0">Nier: Automata Game Of The Yorha Edition
13,Nome do game:      Death Stranding
14,Nome do game:      Resident Evil 2
15,Nome do game:      Days Gone
16,Nome do game:      Shadow Of The Colossus
17,Nome do game:      Uncharted 4: A Thief's End
18,Nome do game:      Cyberpunk 2077


IndexError: list index out of range

In [ ]:
df.to_csv(r"E:/Projeto_Python/Bases/dfgames_preco3.csv", header=True, sep = ";",index = False)

In [ ]:
links_fim2.iloc[1102,0]